In [ ]:
# GRPH
import networkx as nx

def read_fasta(file_path):
    sequences = {}
    with open(file_path, 'r') as file:
        label  = None
        for line in file:
            line = line.strip()
            if line.startswith(">"): 
                label = line[1:]
                sequences[label] = ""
            else:
                sequences[label] += line

    return sequences

# Creating a directed graph:
def create_overlap_graph(sequences, k):

# initializing an empty directed graph:
    graph = nx.DiGraph()     
                                                            
    for label1, seq1 in sequences.items():               
        suffix = seq1[-k:]                                  # slicing the sequence from the last k characters
        for label2, seq2 in sequences.items():              
            if label1 != label2:                            # no self-loops            
                prefix = seq2[:k]                           # slicing the sequence to get the first k characters
                if suffix == prefix:                        
                    graph.add_edge(label1, label2)          # adding directed edge from label1 to label2 
    return graph


fasta_file = "rosalind_grph.txt"  
sequences = read_fasta(fasta_file)

k = 3

overlap_graph = create_overlap_graph(sequences, k)

for edge in overlap_graph.edges():
    print(f"{edge[0]} {edge[1]}")


In [ ]:
# TREE

def read_graph(filename):
    adj_list = {}  
    
    with open(filename, 'r') as file:
        n = int(file.readline().strip())         

        for i in range(1, n + 1):
            adj_list[i] = []
        
        for line in file:
            u, v = map(int, line.strip().split())  
            adj_list[u].append(v)
            adj_list[v].append(u)                  # since the graph is undirected, we add v to the list of neighbors for u, and u to the list of neighbors for v
    return n, adj_list                           

# Recursive DFS to visit all nodes in a component: 
def dfs(node, visited, adj_list):
    visited[node] = True                           # so we don't visit it again 
    for neighbor in adj_list[node]:          
        if not visited[neighbor]:                  # if a neighbor hasn't been visited yet, recursively calling dfs() on that neighbor
            dfs(neighbor, visited, adj_list) 

# Calculating the minimum number of edges required to make the graph a connected tree:
def min_edges_to_add(n, adj_list):
    visited = [False] * (n + 1)          
    num_components = 0    

    # iterating through each node to find unvisited components:
    for node in range(1, n + 1):
        if not visited[node]:              
            dfs(node, visited, adj_list)    
            num_components += 1              
    
    return num_components - 1                      # connecting all components into one tree


n, adj_list = read_graph("rosalind_tree.txt")  
result = min_edges_to_add(n, adj_list)                       
print(result)


In [ ]:
# LONG

def read_fasta(file_path):

    sequences = []
    with open(file_path, 'r') as file:
        current_sequence = ""
        for line in file:
            line = line.strip()
            if line.startswith(">"):
                if current_sequence:
                    sequences.append(current_sequence)
                current_sequence = ""
            else:
                current_sequence += line
        if current_sequence: 
            sequences.append(current_sequence)
    return sequences

# Maximum overlap between the suffix of s1 and the prefix of s2:
def find_overlap(s1, s2):

    max_overlap = 0                 
    merged = s1 + s2                #  simple concatenation of s1 and s2
    
    for i in range(1, len(s1) + 1): 
        if s1[-i:] == s2[:i]:       
            max_overlap = i        
            merged = s1 + s2[i:]    # containing all of s1 and appending only the non-overlapping part of s2 

    return max_overlap, merged

# Constructing shortest superstring:
def shortest_superstring(strings):
  
    while len(strings) > 1:
        max_overlap = -1       # to ensure any overlap found will be larger
        best_pair = (0, 0)
        merged_string = ""

        for i in range(len(strings)):
            for j in range(len(strings)):
                if i != j:                                                         
                    overlap_length, merged = find_overlap(strings[i], strings[j]) 
                    if overlap_length > max_overlap:                              
                        max_overlap = overlap_length    # updating max_overlap to the new overlap length
                        best_pair = (i, j)                                        
                        merged_string = merged                                  

        # creating new list with the merged string replacing the best pair:
        i, j = best_pair
        new_strings = []  

        for k in range(len(strings)):             
            if k == i:
                new_strings.append(merged_string) 
            elif k != j:
                new_strings.append(strings[k])  
        strings = new_strings  

    return strings[0]


file_path = "rosalind_long.txt" 
strings = read_fasta(file_path)
result = shortest_superstring(strings)
print(result)


In [ ]:
# DEG

def read_graph(filename):
    with open(filename, 'r') as file:
        n, m = map(int, file.readline().split())  
        
        # initializing the degree array with 0s for each vertex:
        degrees = [0] * (n + 1)
    
        for line in file:
            u, v = map(int, line.split())
            degrees[u] += 1 
            degrees[v] += 1  
        
    return degrees[1:]  # new list 

filename = "rosalind_deg.txt"  
degrees = read_graph(filename)
print(" ".join(map(str, degrees)))

In [ ]:
# DDEG

def read_graph(filename):
    with open(filename, 'r') as file:
        n, m = map(int, file.readline().split())  
        
        adj_list = {}            # adjacency list for each vertex
        degrees = [0] * (n + 1)  # degrees of vertices (1-indexed)

        
        for i in range(1, n + 1):
            adj_list[i] = [] # initializing an empty list for each vertex in the adj_list:
        
        for line in file:
            u, v = map(int, line.split())  
            adj_list[u].append(v) 
            adj_list[v].append(u)  
            degrees[u] += 1  
            degrees[v] += 1  

    return n, adj_list, degrees

# Sum of the degrees of the neighbors for each vertex: 
def sum_of_neighbor_degrees(n, adj_list, degrees):

    result = [0] * (n + 1)    # to store the sum

    for i in range(1, n + 1):
        total_sum = 0
        for neighbor in adj_list[i]:  
            total_sum += degrees[neighbor]  
        result[i] = total_sum 
    return result[1:]


filename = "rosalind_ddeg.txt" 
n, adj_list, degrees = read_graph(filename)  
result = sum_of_neighbor_degrees(n, adj_list, degrees) 
print(" ".join(map(str, result)))  


In [ ]:
# CC
import networkx as nx

def read_graph(filename):
    adj_list = {}                              
    with open (filename, 'r') as file:
        n,m = map(int, file.readline().split()) 
        for i in range(1, n+1):                 
            adj_list[i] = []                   
    # for each edge read its two end points:
        for i in range(m):                                 
            u,v = map(int, file.readline().split())   
            adj_list[u].append(v)                          
            adj_list[v].append(u)                           
    return n, adj_list

# Performing Depth-Searc to explore all the nodes of the connected component system, starting from a given node: 
def dfs(node, visited, adj_list):
    visited[node] = True                         
    for neighbour in adj_list[node]:            
        if not visited[neighbour]:               
            dfs(neighbour, visited, adj_list)    
                                                
# Counting the connected components in the graph:
def count_connected_components(n, adj_list):
    visited = [False] * (n+1)                    # created a list to track visited nodes:
    components = 0                              

    for node in range(1, n+1):                   
        if not visited[node]:                    # if node is not visited it means it belongs to a new component:
            components += 1                    
            dfs(node, visited, adj_list)    
    
    return components                           
filename = "rosalind_cc.txt"
n, adj_list = read_graph(filename)
components = count_connected_components(n, adj_list)
print(components)


In [ ]:
# BFS

def read_graph(filename):
    adj_list = {}
    with open(filename, 'r') as file:
        n, m = map(int, file.readline().split())  
        
        for i in range(1, n + 1):
            adj_list[i] = []
        
        for i in range(m):                            
            u, v = map(int, file.readline().split()) 
            adj_list[u].append(v)                   
        
    return n, adj_list  


def bfs_shortest_path(n, adj_list):
    
    D = [-1] * (n + 1)  # distanced array is intially set -1, meaning unreachable
    D[1] = 0            # (starting point)
    
    to_visit = [1]  # starting with node 1 

    index = 0                      
    while index < len(to_visit):
        current = to_visit[index]  
        index += 1               

        for neighbor in adj_list[current]:  # exploring all neighbors of the current node:
            if D[neighbor] == -1:             
                D[neighbor] = D[current] + 1  # setting the distance
                to_visit.append(neighbor)     

    return D[1:]


n, adj_list = read_graph('rosalind_bfs.txt')  
distances = bfs_shortest_path(n, adj_list)  
print(*distances)  


In [ ]:
# DIJ

import networkx as nx

def read_graph(filename):
    G = nx.DiGraph()                                             
    with open (filename,'r') as file: 
        n, m = map(int, file.readline().split())               

        for i in range (m):                                     
            u, v, weight = map(int, file.readline().split())     # u and v are the nodes, weight is the weight of the edge
            G.add_edge(u, v, weight=weight)                      # adding directed edge from node u to node v with the specified weight to the graph
    
    return n, G

# Dijkstra algorithm to find the shortest path from node 1:
def dijkstra_shortest_paths(n,G):

    distances = nx.shortest_path_length(G,source = 1, weight = 'weight')    
    D = [-1] * (n+1)                                                        # initialized all the distances to -1 so that if a node is not reachable, it will remain as -1

    for node, dist in distances.items():                                    
        D[node] = dist                                                      # assigned the shortest path dist to the result array D 

    return D

filename = 'rosalind_dij.txt'
n, G = read_graph(filename)

D = dijkstra_shortest_paths(n, G)
print(*D[1:])                                                               



In [ ]:
# DAG
import networkx as nx

def read_graph(filename):
    with open(filename, 'r') as file:
        lines = [line.strip() for line in file if line.strip()]  
    k = int(lines[0])  # number of graphs
    results = []      

    line = 1  
    
    for i in range(k): 
        if line >= len(lines):
            print("Error: Unexpected end of file. Each graph must start with a line specifying nodes and edges.")
            return []

        n, m = map(int, lines[line].split()) 
        line += 1                           
        G = nx.DiGraph()  # created a new directed graph to store the nodes and edges for the current graph being processed

        
        for i in range(m):                                                                   
            if line >= len(lines):                                                          
                print("Error: Unexpected end of file. Edges are missing for a graph.")
                return []

            u, v = map(int, lines[line].split())   # (lines[line]) is expected to contain two integers u and v which represent the source and target nodes of the directed edge:              
            G.add_edge(u, v)                                                                  
            line += 1                                                                         

        # checking if the graph is acyclic:
        if nx.is_directed_acyclic_graph(G):
            results.append("1")
        else:
            results.append("-1")

    return results


filename = "rosalind_dag.txt"  
results = read_graph(filename)
print(' '.join(results))



In [ ]:
# BIP

# Exploring the graph and trying to color it with two colors to check bipartiteness:
def dfs(graph, node, color, colors):
    colors[node] = color  
    for neighbor in graph[node]:
        if colors[neighbor] == -1:  # if the neighbor is not colored:
            if not dfs(graph, neighbor, 1 - color, colors): # color with the opposite color
                return False
        elif colors[neighbor] == colors[node]:  # if the neighbor has the same color, not bipartite:
            return False
    return True

def read_graph(filename):
    with open(filename, 'r') as file:
        k = int(file.readline())  
        results = []

        for i in range(k):
            line = file.readline().strip()
            while not line: 
                line = file.readline().strip()

            
            line_split = line.split()
            if len(line_split) != 2:  
                print("Error: Invalid line format for n and m.")
                return []

            n, m = map(int, line_split) 

            graph = []  # empty list to hold the adjacency list:
            for i in range(n):
                graph.append([]) 

            for i in range(m):
                line = file.readline().strip()
                while not line:  
                    line = file.readline().strip()

                line_split = line.split()
                if len(line_split) != 2:
                    print(f"Error: Invalid edge format in line: {line}")
                    return []

                u, v = line_split
                u, v = int(u), int(v)

                # Checking that u and v are within the valid range of nodes:
                if u <= 0 or v <= 0 or u > n or v > n:
                    print(f"Error: Edge ({u}, {v}) refers to nodes that are out of range (1 to {n}).")
                    return []

                graph[u - 1].append(v - 1)
                graph[v - 1].append(u - 1)  

            
            colors = [-1] * n   # array to track the color of each node:
            is_bipartite = True

            for node in range(n):
                if colors[node] == -1: 
                    if not dfs(graph, node, 0, colors): 
                        is_bipartite = False
                        break

            results.append("1" if is_bipartite else "-1")

    return results

filename = "rosalind_bip.txt"  
results = read_graph(filename)
print(' '.join(results))



In [ ]:
# BF

# Computing the shortest distances from a source node to all other nodes:
def bellman_ford(n, edges):
    
    distances = [float('inf')] * n     # initializing distances with infinity, except the source node (node 1):
    distances[0] = 0                   # distance to the source node is 0

    for i in range(n - 1):
        for u, v, weight in edges:     # checking if the distance to node v can be shortened by going through node u:
            if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                distances[v] = distances[u] + weight
    
    result = []
    for d in distances:                 # if a node’s distance is still inf, it's unreachable from the source node (node 1)
        if d == float('inf'):         
            result.append('x')
        else:
            result.append(str(d))
    
    return result

def read_graph(filename):
    with open(filename, 'r') as file:
        n, m = map(int, file.readline().split())  
        edges = []
        
        for i in range(m):
            u, v, weight = map(int, file.readline().split())  
            edges.append((u - 1, v - 1, weight))  # converting to 0-based indexing  

        return bellman_ford(n, edges)

filename = "rosalind_bf.txt" 
results = read_graph(filename)
print(" ".join(results))

In [ ]:
# CTE

# Using dijkstra to find the shortest path from a start vertex (using simple lists):
def dijkstra(graph, start, n): 

    dist = [float('inf')] * n  # initially all vertices are unreachable except the starting vertex itself
    dist[start] = 0
    visited = [False] * n     
    
    for i in range(n):         # finding the unvisited vertex with the smallest distance:
        min_dist = float('inf')
        u = -1 

        for i in range(n):
            if not visited[i] and dist[i] < min_dist:
                min_dist = dist[i]
                u = i
        
        if u == -1:  # no more vertices to process
            break
        
        visited[u] = True   
        
        for v, weight in graph[u]:      # checking if we can reach the neighbor with a shorter path than the current known path
            if dist[u] + weight < dist[v]:
                dist[v] = dist[u] + weight
    return dist

# Finding the shortest cycle: 
def find_cycle(n, edges, u, v, edge_weight):
    
    graph = [[] for i in range(n)]      
    for x, y, w in edges:
        if x != u or y != v:
            graph[x].append((y, w))
    
    dist_from_v = dijkstra(graph, v, n)  # running dijkstra from v to find the shortest path to u:
    if dist_from_v[u] != float('inf'):            
        return dist_from_v[u] + edge_weight
    
    return -1  # no cycle 

def process_graphs(filename):
    with open(filename, 'r') as file:
        k = int(file.readline().strip())  
        results = []  
        for i in range(k):
            n, m = map(int, file.readline().split())  

            edges = []
            for i in range(m):
                u, v, w = map(int, file.readline().split())
                edges.append((u - 1, v - 1, w))  

            # first edge is the one we need to check for the cycle:
            u, v, edge_weight = edges[0]
    
            result = find_cycle(n, edges, u, v, edge_weight)

            results.append(str(result))
        print(" ".join(results))

filename = "rosalind_cte.txt"
process_graphs(filename)



In [ ]:
# NWC

# Processessing each graph one by one and determininng if it contains a negative weight cycle:
def detect_negative_cycle(k, graphs):  
    results = []
    
    for edges, n in graphs:
        super_source = n    # ensuring all vertices in the graph are reachable

        for i in range(n): 
            edges.append((super_source, i, 0)) # creating connection from the super_source to every other vertex:
        
      
        distance = [float('inf')] * (n + 1)
        distance[super_source] = 0
        
        # Bellman-Ford Algorithm:
        for i in range(n):  
            for u, v, w in edges: 
                if distance[u] != float('inf') and distance[u] + w < distance[v]:
                    distance[v] = distance[u] + w   
        
        # checking for negative weight cycle:
        has_negative_cycle = False
        for u, v, w in edges:
            if distance[u] != float('inf') and distance[u] + w < distance[v]:  # if any edge still allows a shorter distance, it's a negative weight cycle:
                has_negative_cycle = True
                break
        
        results.append(1 if has_negative_cycle else -1) 
    
    return results

def read_input_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    
    k = int(lines[0].strip())  
    graphs = []
    index = 1
    
    for i in range(k):
        n, m = map(int, lines[index].strip().split()) 
        index += 1
        edges = []
        for _ in range(m):
            u, v, w = map(int, lines[index].strip().split())
            edges.append((u - 1, v - 1, w))  
            index += 1
        graphs.append((edges, n))
    
    return k, graphs

# Main function:
def process_file(file_path):
    k, graphs = read_input_from_file(file_path)
    results = detect_negative_cycle(k, graphs)
    print(" ".join(map(str, results)))

file_path = "rosalind_nwc.txt"  
process_file(file_path)



In [ ]:
# SDAG

# Finding the shortest path:
def shortest_path_dag(n, edges):
    adj_list = [[] for i in range(n)] 
    for u, v, w in edges:
        adj_list[u - 1].append((v - 1, w)) 

    visited = [False] * n  
    topo_order = []

# Performing topological sorting using DFS:
    def dfs(v):          
        visited[v] = True
        for neighbor,i in adj_list[v]:
            if not visited[neighbor]:
                dfs(neighbor)
        topo_order.append(v)   
    for i in range(n):
        if not visited[i]:
            dfs(i)

    topo_order.reverse()  # reversing topo_order list to get the correct topological order

 
    distances = ["x"] * n        # vertices are initially unreachable
    distances[0] = 0             # shortest path from vertex 1 to itself is always zero

    for u in topo_order:                     
        if distances[u] != "x":  
            for v, w in adj_list[u]:                
                if distances[v] == "x" or distances[u] + w < distances[v]: 
                    distances[v] = distances[u] + w

    return distances 

def read_input_from_file(filename):
    with open(filename, 'r') as file:
        n, m = map(int, file.readline().split())
        edges = [tuple(map(int, line.split())) for line in file]  
    return n, edges

# Main function:
def process_file(file_path):
    n, edges = read_input_from_file(file_path)
    result = shortest_path_dag(n, edges)
    print(" ".join(str(x) for x in result))

file_path = "rosalind_sdag.txt"  
process_file(file_path)


